In [ ]:
#参考サイト
#http://rikoubou.hatenablog.com/entry/2019/03/26/143556
#https://kamino.hatenablog.com/entry/opencv_calibrate_camera

In [1]:
#
#カメラのゆがみを戻すための値を計算する
#

In [2]:
import numpy as np
import cv2
import glob
from time import sleep
from datetime import datetime

In [3]:
TMP_FOLDER_PATH = "./temp/"
MTX_PATH = TMP_FOLDER_PATH + "mtx.csv"
DIST_PATH = TMP_FOLDER_PATH + "dist.csv"

In [4]:
def main():
    calcCamera()

#カメラのゆがみを計算する関数
def calcCamera():
    square_size = 25.0 #正方形のサイズ(mm)
    pattern_size = (7, 7)
    #チェスボード(X,Y,Z)座標の指定(Z=0)
    pattern_points = np.zeros((np.prod(pattern_size), 3), np.float32)
    pattern_points[:,:2] = np.indices(pattern_size).T.reshape(-1,2)
    pattern_points *= square_size
    obj_points = []
    img_points = []

    for fn in glob.glob("./img/*"):
        #画像の取得
        im = cv2.imread(fn,0)
        print("loading..." + fn)
        #チェスボードのコーナーの検出
        found, corner = cv2.findChessboardCorners(im, pattern_size)
        #コーナーがあれば
        if found:
            term = (cv2.TERM_CRITERIA_EPS + cv2.TermCriteria_COUNT, 30, 0.1)
            corners2 = cv2.cornerSubPix(im, corner, (5,5), (-1, -1), term)
            #マークを付けて画像保存
            img = cv2.drawChessboardCorners(im, pattern_size, corners2, found)
            saveImgByTime(TMP_FOLDER_PATH, img)
            sleep(1)
        #コーナーがない場合のエラー処理
        if not found:
            print('chessboard not found')
            continue
        #appendメソッド：リストの最後に引数のオブジェクトを追加
        img_points.append(corner.reshape(-1, 2))
        obj_points.append(pattern_points)
    
    #内部パラメータを計算
    rms, K, d, r, t = cv2.calibrateCamera(obj_points, img_points, (im.shape[1], im.shape[0]), None, None)
    
    #計算結果を表示
    print("RMS = ", rms)
    print("K = \n", K)
    print("d = ", d.ravel())

    #ファイル保存
    saveCalibrationFile(K, d, MTX_PATH, DIST_PATH)

#キャリブレーションCSVファイルを上書き保存する関数
def saveCalibrationFile(mtx, dist, mtx_path, dist_path):
    #カメラ行列の保存
    np.savetxt(mtx_path, mtx, delimiter = ',', fmt="%0.14f")
    #歪み係数の保存
    np.savetxt(dist_path, dist, delimiter = ',', fmt="%0.14f")

#画像を時刻で保存する関数
def saveImgByTime(dirPath, img):
    # 時刻を取得
    date = datetime.now().strftime("%Y%m%d_%H%M%S")
    path = dirPath + date + ".png"
    cv2.imwrite(path, img) # ファイル保存
    print("saved:", path)

if __name__== '__main__':
    main()

loading..../img\20200829_140946.png
saved: ./temp/20200829_141748.png
loading..../img\20200829_140952.png
saved: ./temp/20200829_141749.png
loading..../img\20200829_140957.png
saved: ./temp/20200829_141750.png
loading..../img\20200829_141003.png
saved: ./temp/20200829_141751.png
loading..../img\20200829_141015.png
saved: ./temp/20200829_141752.png
loading..../img\20200829_141022.png
saved: ./temp/20200829_141753.png
loading..../img\20200829_141026.png
saved: ./temp/20200829_141754.png
loading..../img\20200829_141031.png
saved: ./temp/20200829_141755.png
loading..../img\20200829_141041.png
saved: ./temp/20200829_141756.png
loading..../img\20200829_141047.png
saved: ./temp/20200829_141758.png
loading..../img\20200829_141103.png
saved: ./temp/20200829_141759.png
loading..../img\20200829_141117.png
saved: ./temp/20200829_141800.png
loading..../img\20200829_141123.png
saved: ./temp/20200829_141801.png
loading..../img\20200829_141135.png
saved: ./temp/20200829_141802.png
loading..../img\2020